# C-SWM 

In [ ]:
using HDF5
using Knet
using Statistics: mean,std
using Random
include("utils.jl");
import Base: iterate, length

In [ ]:
#Params
SAVE_FOLDER = "./checkpoints"
NUM_STEPS = 1
TRAIN_DATASET_PATH = "/home/cagan/dev/datasets/C-SWM/shapes_train.h5"
BATCH_SIZE = 100
SEED = 0
NUM_OBJECTS = 5

In [ ]:
dtrn = buildDataset(TRAIN_DATASET_PATH, true, BATCH_SIZE);

In [ ]:
for  (k, (obs, action, next_obs)) in enumerate(dtrn)
    
    #Train
    println(obs)
    
end

# C-SWM

### EncoderCNNSmall

TODO: Add bias

In [ ]:
struct EncoderCNNSmall
   
    weights
    bn_vars
    act_fn
    act_fn_hid
    
end

function initEncoderCNNSmall(input_dim, hidden_dim, num_objects, act_fn, act_fn_hid)
    
    weights = Any[param(10,10,3,hidden_dim), param(1,1,hidden_dim,num_objects)]
    bn_vars = Any[bnmoments(), atype(bnparams(hidden_dim))]
    
    return EncoderCNNSmall(weights, bn_vars, act_fn, act_fn_hid)
    
end

function (e_cnn::EncoderCNNSmall)(x)
    
    #w(w_x,w_y,in_ch,out_ch)
    x1 = conv4(e_cnn.weights[1],x; stride = 10)
    x2 = e_cnn.act_fn_hid.(batchnorm(x1, e_cnn.bn_vars[1], e_cnn.bn_vars[2]))
    y = e_cnn.act_fn.(conv4(e_cnn.weights[2],x2;stride=1))
    
    return y
    
end

In [ ]:
obj_extractor = initEncoderCNNSmall(3,64,6,sigm,relu)

In [ ]:
masks = obj_extractor(obs)

### LayerNorm Layer (Taken from: https://github.com/denizyuret/Knet.jl/issues/492)

In [ ]:
struct LayerNorm; a; b; ϵ; end

function LayerNorm(dmodel; eps=1e-6)
    a = param(dmodel; init=ones)
    b = param(dmodel; init=zeros)
    return LayerNorm(a, b, eps)
end

function (l::LayerNorm)(x, o...)
    μ = mean(x,dims=1)
    σ = std(x,mean=μ,dims=1)
    return l.a .* (x .- μ) ./ (σ .+ l.ϵ) .+ l.b                                                         
end

In [ ]:
struct EncoderMLP
    
    weights
    biases
    layer_norm
    act_fn

end


function initEncoderMLP(input_dim, hidden_dim, output_dim, num_objects, act_fn)
    
    weights = [param(hidden_dim,input_dim),param(hidden_dim, hidden_dim),param(output_dim,hidden_dim )]
    biases =  [param0(hidden_dim), param0(hidden_dim), param0(output_dim)]
    layer_norm = LayerNorm(hidden_dim)
    
    return EncoderMLP(weights, biases, layer_norm, act_fn)
    
end

function (e_mlp::EncoderMLP)(x)
    
    dims = size(x)
    println(dims)
    x0 = reshape(x, dims[1]*dims[2], dims[3]*dims[4])
    println(size(x0))
    x1 = e_mlp.act_fn.(e_mlp.weights[1] * x0 .+ e_mlp.biases[1])
    println(size(x1))
    x2 = e_mlp.weights[2]* x1 .+ e_mlp.biases[2]
    println(size(x2))
    x3 = e_mlp.act_fn.(e_mlp.layer_norm(x2))
    println(size(x3))
    x4 = e_mlp.act_fn.(e_mlp.weights[3]*x3 .+ e_mlp.biases[3])
    x5 = reshape(x4, :, dims[3],dims[4])
    
    return x5
    
end

In [ ]:
# Input dim = 25
# Hidden dim = 512
# Output dim = 2

obj_encoder = initEncoderMLP(25,512,2,5,relu)

In [ ]:
o = obj_encoder(masks)

In [ ]:
struct EdgeMLP
    
    weights
    biases
    layer_norm
    act_fn
  
end


function initEdgeMLP(input_dim, hidden_dim, act_fn)
    
    weights = [param(hidden_dim,input_dim*2),param(hidden_dim, hidden_dim),param(hidden_dim,hidden_dim)]
    biases =  [param0(hidden_dim), param0(hidden_dim), param0(hidden_dim)]    
    layer_norm = LayerNorm(hidden_dim)
    
end


function (edge_mlp::EdgeMLP)(x)
    
    x1 = e_mlp.act_fn.(e_mlp.weights[1] * x .+ e_mlp.biases[1])
    println(size(x1))
    x2 = e_mlp.weights[2]* x1 .+ e_mlp.biases[2]
    println(size(x2))
    x3 = e_mlp.act_fn.(e_mlp.layer_norm(x2))
    println(size(x3))
    x4 = e_mlp.act_fn.(e_mlp.weights[3]*x3 .+ e_mlp.biases[3])
    
    return x4
    
end


In [ ]:
struct NodeMLP
    
    weights
    biases
    layer_norm
    act_fn
    
end

function initNodeMLP(node_input_dim, hidden_dim, act_fn, input_dim)
    
    weights = [param(hidden_dim,node_input_dim),param(hidden_dim, hidden_dim),param(input_dim,hidden_dim )]
    biases =  [param0(hidden_dim), param0(hidden_dim), param0(input_dim)]    
    layer_norm = LayerNorm(hidden_dim)
    
    return NodeMLP(weights, biases, layer_norm, act_fn)
    
end


function (node_mlp::NodeMLP)(x)
    
    x1 = e_mlp.act_fn.(e_mlp.weights[1] * x .+ e_mlp.biases[1])
    println(size(x1))
    x2 = e_mlp.weights[2]* x1 .+ e_mlp.biases[2]
    println(size(x2))
    x3 = e_mlp.act_fn.(e_mlp.layer_norm(x2))
    println(size(x3))
    x4 = e_mlp.act_fn.(e_mlp.weights[3]*x3 .+ e_mlp.biases[3])
    
    return
    
end

In [ ]:
struct TransitionGNN
    
    edge_mlp
    node_mlp
    
    ignore_action
    copy_action
    action_dim
    
    edge_list
    batch_size
    
end

function initTransitionGNN(input_dim, hidden_dim, action_dim, num_objects, ignore_action, copy_action, act_fn)
    
    if ignore_action
        
        action_dim = 0
        
    end
    
    #Edge MLP
    edge_mlp = initEdgeMLP(input_dim, hidden_dim, act_fn)
    node_input_dim = hidden_dim + input_dim + action_dim
    
    #Node MLP
    node_mlp = initNodeMLP(node_input_dim, hidden_dim, act_fn, input_dim)
    
    edge_list = Any[]
    batch_size = 0
    
    return TransitionGNN(edge_mlp, node_mlp, ignore_action, copy_action, action_dim, edge_list, batch_size)
    
end


function get_edge_list_fully_connected()
    
end

function (t_gnn::TransitionGNN)(x)
    
    
    #Todo forward prop
    dimensions = size(x)
    batch_size = dimensions[1]
    num_nodes = dimensions[2]
    
    node_attr  = 
    
    if num_nodes > 1
        
    end
    
    if !t_gnn.ignore_action
        
        if t_gnn.copy_action
            
            action_vec = toOneHot(action_dim, )
            
        else
            
            
            
        end
        
    end    
    
end